In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# -----------------------------
# ✅ Depthwise Separable Conv Block (Xception 핵심)
# -----------------------------
class Xception1DBlock(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size=3, stride=1):
        super().__init__()
        self.depthwise = layers.DepthwiseConv1D(
            kernel_size=kernel_size, strides=stride, padding='same')
        self.pointwise = layers.Conv1D(filters, kernel_size=1, padding='same')
        self.bn = layers.BatchNormalization()
        self.relu = layers.ReLU()
        self.skip = None

    def build(self, input_shape):
        in_channels = input_shape[-1]
        if in_channels != self.pointwise.filters:
            self.skip = layers.Conv1D(self.pointwise.filters, 1, padding='same')

    def call(self, x, training=False):
        residual = x
        y = self.depthwise(x)
        y = self.pointwise(y)
        y = self.bn(y, training=training)
        if self.skip is not None:
            residual = self.skip(residual)
        y = layers.add([y, residual])
        return self.relu(y)

# -----------------------------
# ✅ 전체 Xception-1D 모델
# -----------------------------
def build_xception1d(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)

    x = layers.Conv1D(64, 3, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)

    x = Xception1DBlock(64)(x)
    x = Xception1DBlock(128)(x)
    x = Xception1DBlock(256)(x)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

# -----------------------------
# ✅ UCI HAR 데이터 로드
# -----------------------------
def load_har_data(base_path="/content/drive/MyDrive/Colab Notebooks/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/"):
    X_train = pd.read_csv(base_path + "train/X_train.txt", sep=r'\s+', header=None)
    y_train = pd.read_csv(base_path + "train/y_train.txt", sep=r'\s+', header=None)[0]
    X_test = pd.read_csv(base_path + "test/X_test.txt", sep=r'\s+', header=None)
    y_test = pd.read_csv(base_path + "test/y_test.txt", sep=r'\s+', header=None)[0]
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = load_har_data()

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)
num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# (B, F) → (B, F, 1)
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

# -----------------------------
# ✅ 모델 생성 및 학습
# -----------------------------
model = build_xception1d((X_train.shape[1], 1), num_classes)
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

print(model.summary())

device = tf.test.gpu_device_name()
print(f"GPU: {device if device else 'CPU 사용 중'}")

history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=64,
    validation_split=0.2,
    shuffle=True,
    verbose=1
)

# -----------------------------
# ✅ 평가 및 시각화
# -----------------------------
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Test Accuracy: {acc:.4f}")

# 학습 곡선
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Val')
plt.title('Accuracy over Epochs')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Val')
plt.title('Loss over Epochs')
plt.legend()
plt.show()

# 혼돈행렬
y_pred = model.predict(X_test, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)
cm = confusion_matrix(y_true_classes, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap="Blues", xticks_rotation='vertical')
plt.title("Confusion Matrix")
plt.show()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 561, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 561, 64)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 561, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ xception1d_block                │ (None, 561, 64)        │         4,672 │
│ (Xception1DBlock)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ xception1d_block_1              │ (None, 561, 128)       │        17,408 │
│ (Xception1DBlock)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ xception1d_block_2              │ (None, 561, 256)       │        67,584 │
│ (Xception1DBlock)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 123,846 (483.77 KB)

 Trainable params: 122,822 (479.77 KB)

 Non-trainable params: 1,024 (4.00 KB)

None
GPU: CPU 사용 중
Epoch 1/20
80/92 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.4213 - loss: 1.2212

KeyboardInterrupt: 